# MX Youtube Trending Videos Analysis
This notebook will load the Mexican
Youtube Trending Videos Dataset from `data` folder, and then compute and save to file in the `results` folder the following variables:
- Number of videos, channels and categories
- Mean and standard deviation of the number of views, comments, likes and dislikes
- Most popular categories

First we import useful libraries and functions, then we load data from `data` folder in a DataFrame.

In [3]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.types._

val country = "MX"
val numCountries = 2
val df = spark.read.format("csv").option("header", "true").load("data/" + country + "videos_new.csv")

country = MX
numCountries = 2
df = [video_id: string, trending_date: string ... 14 more fields]


[video_id: string, trending_date: string ... 14 more fields]

Visualize the first two lines of the DataFrame (including header and the first data point).

In [4]:
df.show(2)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|SbOwzAl9ZfQ|     17.14.11|Capítulo 12 | Mas...|     MasterChef 2017|         24|2017-11-13T06:06:...|"MasterChef Junio...|310130| 4182|     361|         1836|https://i.ytimg.c...|            FALSE|           FALSE|     

Then we compute the number of channels, videos and categories.

In [5]:
val nChannels = df.select("channel_title").distinct.count
val nVideos = df.count
val nCategories = df.select("category_id").distinct.count


nChannels = 6978
nVideos = 43819
nCategories = 17


17

Here we compute the mean and standard deviation of the number of views, comments, likes and dislikes.

We use the function `df.describe()` then transform the results from `double` to `integer` type, in order to remove the decimals.
Finally we filter out the values of max, min and number of videos, which are not useful.

In [6]:
// Compute mean, stddev, max and min values of likes, dislikes, views and comment_count
val numDataDouble = df.describe("views", "comment_count", "likes", "dislikes")

val numData = numDataDouble.withColumn("views", col("views").cast(IntegerType))
                           .withColumn("likes", col("likes").cast(IntegerType))
                           .withColumn("dislikes", col("dislikes").cast(IntegerType))
                           .withColumn("comment_count", col("comment_count").cast(IntegerType))
                           .filter($"summary" =!= "max")
                           .filter($"summary" =!= "min")
                           .filter($"summary" =!= "count")
//val numDataLong = numData.select(numData.columns.map(c => col(c).cast(IntegerType)) : _*)
numData.show()

+-------+-------+-------------+-----+--------+
|summary|  views|comment_count|likes|dislikes|
+-------+-------+-------------+-----+--------+
|   mean| 342381|         2039|15861|     747|
| stddev|1714690|        13938|81089|   10953|
+-------+-------+-------------+-----+--------+



numDataDouble = [summary: string, views: string ... 3 more fields]
numData = [summary: string, views: int ... 3 more fields]


[summary: string, views: int ... 3 more fields]

Here we compute the most popular channels by grouping the dataset according to channels, and then ordering the dataset.

In [7]:
val groupedByChannel = df.groupBy("channel_title")
val channelsVideoCount = groupedByChannel.count()
                .withColumnRenamed("count", "num_videos")
                .orderBy(desc("num_videos"))
channelsVideoCount.show(10)

+--------------------+----------+
|       channel_title|num_videos|
+--------------------+----------+
|                null|      3368|
|              Cracks|       186|
|             Badabun|       184|
|      Troom Troom Es|       168|
|           Cracks MX|       162|
|       Las Estrellas|       155|
|Televisa Telenovelas|       149|
|      El Salvador 4K|       145|
|       Tu COSMOPOLIS|       143|
|       Enamorándonos|       136|
+--------------------+----------+
only showing top 10 rows



groupedByChannel = RelationalGroupedDataset: [grouping expressions: [channel_title: string], value: [video_id: string, trending_date: string ... 14 more fields], type: GroupBy]
channelsVideoCount = [channel_title: string, num_videos: bigint]


[channel_title: string, num_videos: bigint]

Here we compute the most popular categories by grouping the dataset according to categories, and then ordering the dataset.

In [8]:
val groupedByCategory = df.groupBy("category_id")
val categoriesVideoCount = groupedByCategory.count()
                .withColumnRenamed("count", "num_videos")
                .orderBy(desc("num_videos"))
categoriesVideoCount.show(10)

+-----------+----------+
|category_id|num_videos|
+-----------+----------+
|         24|     13487|
|         22|      8159|
|         17|      4050|
|         10|      3371|
|       null|      3368|
|         25|      3113|
|         26|      2467|
|         23|      1742|
|          1|      1298|
|         20|       994|
+-----------+----------+
only showing top 10 rows



groupedByCategory = RelationalGroupedDataset: [grouping expressions: [category_id: string], value: [video_id: string, trending_date: string ... 14 more fields], type: GroupBy]
categoriesVideoCount = [category_id: string, num_videos: bigint]


[category_id: string, num_videos: bigint]

Now we load the data contained in the `.json` file, with the category ids, matched with the corresponding category names.

The `.json` file has a complex structure, which makes it necessary to perform many steps to polish the data and obtain what we need as a DataFrame.

In [9]:
// Load categories names file data
val categoriesStruct = spark.read.option("multiline","true")
                .json("data/" + country + "_category_id.json")
// extract the array inside the struct and convert to string
val categoriesArray = categoriesStruct.collect()(0)(1)
                .toString.split(",")
// extract categories names and ids
val names = categoriesArray.filter(x => x.contains("]]")).map(x => x.dropRight(2))
val ids = categoriesArray.filter(x => x.length() < 3)
// create a DataFrame out of category ids and names
val categoriesNames = sc.parallelize(ids zip names).toDF("id", "name")

categoriesNames.show(5)

+---+----------------+
| id|            name|
+---+----------------+
|  1|Film & Animation|
|  2|Autos & Vehicles|
| 10|           Music|
| 15|  Pets & Animals|
| 17|          Sports|
+---+----------------+
only showing top 5 rows



categoriesStruct = [etag: string, items: array<struct<etag:string,id:string,kind:string,snippet:struct<assignable:boolean,channelId:string,title:string>>> ... 1 more field]
categoriesArray = Array(WrappedArray(["XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKmPBggty2mZQ", 1, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Film & Animation]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"", 2, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Autos & Vehicles]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxbknKFVe5Lmg"", 10, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Music]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/HwXKamM1Q20q9BN-oBJavSGkfDI"", 15, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Pets...


Array(WrappedArray(["XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKmPBggty2mZQ", 1, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Film & Animation]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"", 2, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Autos & Vehicles]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxbknKFVe5Lmg"", 10, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Music]], " ["XI7nbFXulYBIpL0ayR_gDh3eu1k/HwXKamM1Q20q9BN-oBJavSGkfDI"", 15, youtube#videoCategory, [true, UCBR8-60-B28hp2BmDPdntcQ, Pets...

Now we can join the two DataFrames of top categories and categories names, obtaining a DataFrame containing only categories names and number of videos belonging to that category.

In [ ]:
// join the dataframes so the categories are matched with their names
val topCategories = categoriesVideoCount.join(categoriesNames, categoriesVideoCount("category_id") === categoriesNames("id"), "leftouter")
                    .orderBy(desc("num_videos"))
                    .select("name", "num_videos")
                    .filter($"name" =!= "null")

// hide the numeric column in the output.
topCategories.show()

Now that we have computed all the data we need, we can create a single-line DataFrame containing the data for this country.

We need to transform DataFrames into Lists, which is a structure that enable us to extract single values.

In [ ]:
val numDataList = numData.rdd.collect().toList
val topCategoriesList = topCategories.rdd.collect().toList

val statsSeq = Seq((country, nChannels.toString, 
        nVideos.toString, nCategories.toString,
        numDataList(0)(1).toString, numDataList(1)(1).toString,
        numDataList(0)(2).toString, numDataList(1)(2).toString,
        numDataList(0)(3).toString, numDataList(1)(3).toString,
        numDataList(0)(4).toString, numDataList(1)(4).toString,
        topCategoriesList(1)(0).toString, topCategoriesList(2)(0).toString,
        topCategoriesList(3)(0).toString, topCategoriesList(4)(0).toString,
        topCategoriesList(5)(0).toString, topCategoriesList(6)(0).toString,
        topCategoriesList(7)(0).toString, topCategoriesList(8)(0).toString,
        topCategoriesList(9)(0).toString, topCategoriesList(10)(0).toString))

val statsRDD = spark.sparkContext.parallelize(statsSeq)
val statsDF = statsRDD.toDF("country", "num_channels", 
                            "num_videos", "num_categories",
                            "views_mean", "views_stddev",
                            "comments_mean", "comments_stddev",
                            "likes_mean", "likes_stddev",
                            "dislikes_mean", "dislikes_stddev",
                            "1_category", "2_category", 
                            "3_category", "4_category",
                            "5_category", "6_category", 
                            "7_category", "8_category",
                            "9_category", "10_category")
statsDF.show()

In [ ]:
Load the data computed previously about other countries, and join it with the data from the current country.

In [ ]:
val allCountriesData = spark.read.format("csv")
                .option("header", "true")
                .load("results/partial_" + numCountries.toString)
val newAllCountriesData = allCountriesData.union(statsDF)

Finally, we save the final DataFrame to file, in the `results` folder.

We use the function `repartition(1)` in order to generate only one file, instead of separating the data into several different files. 

In [ ]:
newAllCountriesData.repartition(1)
             .write
             .format("csv")
             .option("header", "true")
             .save("results/partial_" + (numCountries + 1).toString)